In [1]:
from utils.dataloader import DataLoader
from utils.osmGraph import GraphFromHmlFile
import os
import pandas as pd
import io
import geopandas as gpd
from utils.apiKeys import Censusapi
import requests
import plotly.graph_objects as go
import numpy as np
import plotly


In [2]:
dataFolder = "./data"
graphFileName = "hennepinCounty.graphml"
graph = GraphFromHmlFile(os.path.join(dataFolder, graphFileName))
print(len(graph.edgesGdf))

109283


In [3]:
stationFileName = "stationData_MN.csv"
stationData = DataLoader.readCSV(os.path.join(dataFolder, stationFileName))

In [4]:
stationData.describe()

,Unnamed: 0,ZIP,Latitude,Longitude,EV Level1 EVSE Num,EV Level2 EVSE Num,EV DC Fast Count
count,559.000000,559.000000,559.000000,559.000000,559.000000,559.000000,559.000000
mean,279.000000,55482.341682,45.113029,-93.351857,0.039356,1.930233,0.431127
std,161.513673,405.437592,0.795041,0.799114,0.547287,1.322726,1.535190
min,0.000000,53662.000000,43.633850,-96.767240,0.000000,0.000000,0.000000
25%,139.500000,55124.500000,44.858374,-93.446979,0.000000,1.000000,0.000000
50%,279.000000,55405.000000,44.969550,-93.275995,0.000000,2.000000,0.000000
75%,418.500000,55606.000000,45.078028,-93.088530,0.000000,2.000000,0.000000
max,558.000000,56636.000000,48.054129,-89.692650,12.000000,16.000000,12.000000


In [5]:
stationDataSelected = stationData[stationData['ZIP'] == 55455]

In [6]:
stationDataSelected.index

Int64Index([36, 52, 76, 77, 80, 81, 105, 188], dtype='int64')

In [7]:
stationPositionList = []
for i in stationDataSelected.index:
    stationPositionList.append((stationDataSelected.loc[i, 'Latitude'], stationDataSelected.loc[i, 'Longitude']))

In [8]:
stationPositionList

[(44.974036, -93.230989),
 (44.969677, -93.243328),
 (44.97145, -93.236027),
 (44.976321, -93.23395),
 (44.978214, -93.224562),
 (44.976277, -93.228946),
 (44.973875, -93.230153),
 (44.979167, -93.228946)]

In [9]:
censusFileName = "censusData_MN_Population.csv"
censusData = DataLoader.readCSV(os.path.join(dataFolder, censusFileName))

In [10]:
censusData.describe()

,Unnamed: 0,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,GEOID,ALAND,AWATER,INTPTLAT,INTPTLON,B01001_001E,state,county,tract,block group,geoid
count,4706.000000,4706.0,4706.000000,4706.000000,4706.000000,4.706000e+03,4.706000e+03,4.706000e+03,4706.000000,4706.000000,4706.000000,4706.0,4706.000000,4706.000000,4706.000000,4.706000e+03
mean,2352.500000,27.0,80.320017,204739.733532,2.238207,2.708052e+11,4.382324e+07,4.026745e+06,45.232614,-93.594910,1190.005525,27.0,80.320017,204739.733532,2.238207,2.708052e+11
std,1358.649513,0.0,50.377466,307551.101353,1.143291,5.039194e+08,1.459062e+08,7.060843e+07,0.967383,1.000160,535.489228,0.0,50.377466,307551.101353,1.143291,5.039194e+08
min,0.000000,27.0,1.000000,100.000000,0.000000,2.700177e+11,0.000000e+00,0.000000e+00,43.511983,-97.029791,0.000000,27.0,1.000000,100.000000,0.000000,2.700177e+11
25%,1176.250000,27.0,41.000000,26708.750000,1.000000,2.704145e+11,6.362640e+05,0.000000e+00,44.819986,-93.830478,816.250000,27.0,41.000000,26708.750000,1.000000,2.704145e+11
50%,2352.500000,27.0,53.000000,60739.000000,2.000000,2.705313e+11,1.582888e+06,1.610050e+04,44.984657,-93.307253,1099.000000,27.0,53.000000,60739.000000,2.000000,2.705313e+11
75%,3528.750000,27.0,123.000000,130276.500000,3.000000,2.712304e+11,1.627754e+07,4.741442e+05,45.334613,-93.114646,1481.750000,27.0,123.000000,130276.500000,3.000000,2.712304e+11
max,4705.000000,27.0,173.000000,990100.000000,7.000000,2.717397e+11,3.877442e+09,4.185238e+09,48.973930,-90.016378,4484.000000,27.0,173.000000,990100.000000,7.000000,2.717397e+11


In [11]:
censusDataSelected = censusData[censusData['TRACTCE']==104901]

In [34]:
censusDataSelected

,Unnamed: 0,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,GEOID,NAMELSAD,MTFCC,FUNCSTAT,ALAND,...,INTPTLAT,INTPTLON,geometry,NAME,B01001_001E,state,county,tract,block group,geoid
559,559,27,53,104901,1,270531049011,Block Group 1,G5030,S,421213,...,44.971982,-93.233441,"POLYGON ((-93.239268 44.973257, -93.2388 44.97...","Block Group 1, Census Tract 1049.01, Hennepin ...",4484,27,53,104901,1,270531049011
763,763,27,53,104901,2,270531049012,Block Group 2,G5030,S,908632,...,44.977829,-93.234230,"POLYGON ((-93.243853 44.979307999999996, -93.2...","Block Group 2, Census Tract 1049.01, Hennepin ...",1998,27,53,104901,2,270531049012


In [12]:
censusBlockList = []
for i in censusDataSelected.index:
    censusBlockList.append((censusDataSelected.loc[i, 'INTPTLAT'], censusDataSelected.loc[i, 'INTPTLON']))

In [13]:
censusBlockList

[(44.9719823, -93.2334415), (44.9778291, -93.2342296)]

In [14]:
positionDict = {"station": stationPositionList, "census block group": censusBlockList}

In [35]:
figureFolder = "./results"
figureName = "test.html"
graph.plotData(positionDict, ['green', 'red'] ,os.path.join(figureFolder,figureName))

In [16]:
stationLatList,  stationLonList = zip(*stationPositionList)
stationNodeList,  stationDisToNode = graph.getNearestNode(stationLatList, stationLonList)
stationNodeList,  stationDisToNode

([320363001,
  33822058,
  9421986672,
  33842546,
  34566808,
  261319041,
  261298089,
  34191216],
 [70.46782681917685,
  66.12555583876915,
  98.20830048064921,
  73.64302031497375,
  130.50664577950886,
  91.46854165968767,
  100.56309819822971,
  142.71768466725018])

In [17]:
censusLatList,  censusLonList = zip(*censusBlockList)
censusNodeList,  censusDisToNode = graph.getNearestNode(censusLatList, censusLonList)
censusNodeList,  censusDisToNode

([246710361, 34177358], [62.441805227521264, 73.49242906040053])

In [28]:
distanceToEachStation = {}
pathList = []
for o in censusNodeList:
    disList = []
    for d in stationNodeList:
        path = graph.shortestPath(o, d)
        pathList.append(path)
        disList.append(graph.calPathLength(path))
    distanceToEachStation[o] = disList

In [26]:
distanceToEachStation

{246710361: [1757.195,
  2446.6720000000005,
  199.637,
  1210.4830000000002,
  2013.714,
  1762.9250000000002,
  1556.426,
  1597.385],
 34177358: [843.027,
  2378.996,
  1323.3860000000002,
  263.11899999999997,
  790.1579999999999,
  539.3689999999999,
  642.258,
  373.82899999999995]}

In [33]:
pathList[0]

[246710361,
 246710356,
 1821777933,
 246710346,
 34198362,
 34665658,
 34646309,
 244113404,
 33359705,
 261298089,
 244111945,
 320363001]

In [30]:
graph.nodeList2GPSTraj(pathList[0])

[[44.9724653, -93.2338464],
 [44.972458, -93.2345925],
 [44.9724115, -93.2346998],
 [44.9723859, -93.2348491],
 [44.9723902, -93.2353495],
 [44.9723948, -93.2359434],
 [44.9724257, -93.236043],
 [44.9724811, -93.2361005],
 [44.9724811, -93.2361005],
 [44.9726636, -93.2361744],
 [44.972678, -93.2361802],
 [44.9727526, -93.2362262],
 [44.9727755, -93.2362503],
 [44.9727961, -93.2362718],
 [44.9728286, -93.2363417],
 [44.9728475, -93.2364038],
 [44.9728475, -93.2364038],
 [44.9728436, -93.2365236],
 [44.9728295, -93.2366957],
 [44.9728295, -93.2366957],
 [44.9728097, -93.2367804],
 [44.9728017, -93.236863],
 [44.9727885, -93.2377713],
 [44.9727864, -93.2379178],
 [44.9727864, -93.2379178],
 [44.9726789, -93.2379031],
 [44.9726613, -93.2379007],
 [44.9726441, -93.2378984],
 [44.9724745, -93.2378513],
 [44.9722075, -93.2377546],
 [44.9720807, -93.2376868],
 [44.9720276, -93.2376584],
 [44.9720276, -93.2376584],
 [44.971973, -93.2376373],
 [44.9719641, -93.2376339],
 [44.9719061, -93.2376116

In [32]:
figureName = "testPath.html"
graph.plotTrajList([graph.nodeList2GPSTraj(pathList[0])], os.path.join(figureFolder,figureName))